<a href="https://colab.research.google.com/github/elizabethyoo/team_9/blob/data%2FBFS_mf/Complie_sbp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import glob
import re
import numpy as no
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
files = glob.glob(f"/content/drive/MyDrive/Group 9 Project Documents/Datasets/business_pulse/*.xlsx") 

In [5]:
emp_files = [x for x in files if 'empclass' in x]
st_sector_files = [x for x in files if '/national_state_sector' in x]
st_files = [x for x in files if '/national_state_' in x and not '/national_state_sector' in x]
naics3_files = [x for x in files if 'naics3' in x]
indexes_files = [x for x in files if 'indexes' in x]
msa_files = [x for x in files if 'top_50_msa' in x]
response_urr_files = [x for x in files if 'response_urr' in x]

In [6]:
st_sector_files[0]

'/content/drive/MyDrive/Group 9 Project Documents/Datasets/business_pulse/national_state_sector_26Apr20_02May20.xlsx'

In [7]:
regex = r'\d+[A-Z][a-z]+\d+_\d+[A-Z][a-z]+\d+'  
match = re.search(regex, st_sector_files[0].split('/')[-1])[0]
print(match)

26Apr20_02May20


In [8]:
st_df = pd.read_excel(st_sector_files[0]).assign(DATE=match)

In [9]:
st_df.head()

,ST,NAICS_SECTOR,INSTRUMENT_ID,QUESTION,ANSWER_ID,ANSWER_TEXT,ESTIMATE_PERCENTAGE,SE,DATE
0,-,-,1,"Overall, how has this business been affected b...",1,Large negative effect,51.4%,0.21%,26Apr20_02May20
1,-,-,1,"Overall, how has this business been affected b...",2,Moderate negative effect,38.5%,0.27%,26Apr20_02May20
2,-,-,1,"Overall, how has this business been affected b...",3,Little or no effect,7.6%,0.29%,26Apr20_02May20
3,-,-,1,"Overall, how has this business been affected b...",4,Moderate positive effect,1.7%,0.08%,26Apr20_02May20
4,-,-,1,"Overall, how has this business been affected b...",5,Large positive effect,0.8%,0.06%,26Apr20_02May20


In [10]:
st_df.tail()

,ST,NAICS_SECTOR,INSTRUMENT_ID,QUESTION,ANSWER_ID,ANSWER_TEXT,ESTIMATE_PERCENTAGE,SE,DATE
13111,WY,62,11,"Since March 13, 2020, has this business missed...",2,No,91.3%,9.35%,26Apr20_02May20
13112,WY,62,12,"Since March 13, 2020, has this business missed...",2,No,91.3%,9.19%,26Apr20_02May20
13113,WY,62,13,"Since March 13, 2020, has this business reques...",1,Paycheck Protection Program (PPP),90.2%,10.31%,26Apr20_02May20
13114,WY,62,2,"In the last week, did this business experience...",2,"Yes, decreased",100.0%,0.00%,26Apr20_02May20
13115,WY,62,8,"In the last week, did this business shift to t...",2,No,100.0%,0.00%,26Apr20_02May20


# TO DO

1. The state name is in abbreviation. Data structure/ value for state name is different from the household data. Therefore, a conversion of the state name is needed. (Either on the small business dataset or the household dataset)

2. Select features that can be used as an indicator for business recovery.

3. from the household datasets, find interesting variables that might be associated with the business recovery. 